In [7]:
# 파일 활용 전 전처리 
import pandas as pd

# CSV 파일 읽기, 칼럼 이름이 있는 줄을 지정
try:
    # 'header=[0,1]'을 통해 CSV 파일의 첫 번째와 두 번째 행을 칼럼 이름으로 사용
    df = pd.read_csv('original/recognition/범죄_발생부터_검거까지의_기간_2018_2022.csv', header=[0,1])
except:
    # 파일이 없을 경우 오류메시지 출력
    print("\n해당하는 파일이 없습니다.")
    
# '-' 문자를 0으로 대체
df = df.replace('-', 0)

# 년도별로 데이터를 분리하고 각각의 CSV 파일로 저장
for year in range(2018, 2023):
    # 해당 년도의 데이터 선택
    # 'df.columns.get_level_values(0).str.startswith(str(year))'는 첫 번째 레벨의 칼럼 이름이 해당 년도로 시작하는 칼럼을 선택합니다.
    # 'df.columns.get_level_values(0).isin(['죄종별(1)', '죄종별(2)'])'는 첫 번째 레벨의 칼럼 이름이 '죄종별(1)' 또는 '죄종별(2)'인 칼럼을 선택합니다.
    df_year = df.loc[:, df.columns.get_level_values(0).str.startswith(str(year)) | \
            df.columns.get_level_values(0).isin(['죄종별(1)', '죄종별(2)'])].copy()

    # 해당 년도의 데이터가 있는 경우에만 '3개월초과' 칼럼 계산
    if not df_year.empty:
        # '6개월이내', '1년이내', '1년초과' 칼럼을 '3개월초과' 칼럼으로 합치기
        # 'df_year.xs('6개월이내', level=1, axis=1).sum(axis=1)'는 '6개월이내' 칼럼의 값을 모두 더합니다. 'level=1'은 두 번째 레벨의 칼럼 이름을 사용하라는 의미이며, 'axis=1'은 칼럼을 기준으로 연산하라는 의미입니다.
        df_year.loc[:, (str(year), '3개월초과')] = df_year.xs('6개월이내', level=1, axis=1).sum(axis=1) + df_year.xs('1년이내', level=1, axis=1).sum(axis=1) + df_year.xs('1년초과', level=1, axis=1).sum(axis=1)
        
        # '6개월이내, '1년이내', '1년초과' 칼럼 삭제
        # 'errors='ignore''는 해당 칼럼이 없는 경우에도 오류를 발생시키지 않고 무시하라는 의미입니다.
        df_year = df_year.drop(columns=[(str(year), '6개월이내'), (str(year), '1년이내'), (str(year), '1년초과')], errors='ignore')
        
        # 새로운 CSV 파일로 저장
        # 'index=False'는 인덱스를 CSV 파일에 저장하지 않으라는 의미입니다.
        df_year.to_csv(f'범죄_발생부터_검거까지의_기간_{year}.csv', index=False)



In [9]:
# csv파일 활용 전 전처리

import pandas as pd

# CSV 파일 읽기, 칼럼 이름이 있는 줄을 지정
try:
    df = pd.read_csv('original/recognition/범죄_발생부터_인지까지의_기간_2018_2022.csv', header=[0,1])
except:
    print("\n해당하는 파일이 없습니다.")
# '-' 문자를 0으로 대체
df = df.replace('-', 0)

# 년도별로 데이터를 분리하고 각각의 CSV 파일로 저장
for year in range(2018, 2023):
    # 해당 년도의 데이터 선택
    df_year = df.loc[:, df.columns.get_level_values(0).str.startswith(str(year)) | df.columns.get_level_values(0).isin(['죄종별(1)', '죄종별(2)'])].copy()
    
    # 해당 년도의 데이터가 있는 경우에만 '1일이내' 칼럼 계산
    if not df_year.empty:
        # '1시간이내', '2시간이내', '5시간이내', '12시간이내', '24시간이내' 칼럼을 '1일이내' 칼럼으로 합치기
        df_year.loc[:, (str(year), '1일이내')] = df_year.xs('1시간이내', level=1, axis=1).sum(axis=1) + \
        df_year.xs('2시간이내', level=1, axis=1).sum(axis=1) + df_year.xs('5시간이내', level=1, axis=1).sum(axis=1) + \
        df_year.xs('12시간이내', level=1, axis=1).sum(axis=1) + df_year.xs('24시간이내', level=1, axis=1).sum(axis=1)
        
        # '1시간이내', '2시간이내', '5시간이내', '12시간이내', '24시간이내' 칼럼 삭제
        df_year = df_year.drop(columns=[(str(year), '1시간이내'), (str(year), '2시간이내'), (str(year), '5시간이내'), (str(year), '12시간이내'), (str(year), '24시간이내')], errors='ignore')
        
        # 칼럼 순서 변경
        cols = df_year.columns.tolist()
        cols.insert(cols.index((str(year), '2일이내')), cols.pop(cols.index((str(year), '1일이내'))))
        df_year = df_year.reindex(columns=cols)
        
        # 새로운 CSV 파일로 저장
        df_year.to_csv(f'csv/recognition/범죄_발생부터_인지까지의_기간_{year}.csv', index=False)


In [11]:
# csv파일 활용 전 전처리

import pandas as pd

# CSV 파일 읽기, 칼럼 이름이 있는 줄을 지정
try:
    df = pd.read_csv('original/recognition/범죄의_검거단서_2018_2022.csv', header=[0,1])
except:
    print("\n해당하는 파일이 없습니다.")
# '-' 문자를 0으로 대체
df = df.replace('-', 0)

# 년도별로 데이터를 분리하고 각각의 CSV 파일로 저장
for year in range(2018, 2023):
    # 해당 년도의 데이터 선택
    df_year = df.loc[:, df.columns.get_level_values(0).str.startswith(str(year)) | df.columns.get_level_values(0).isin(['죄종별(1)', '죄종별(2)'])].copy()
        
    # 새로운 CSV 파일로 저장
    df_year.to_csv(f'csv/recognition/범죄의_검거단서_{year}.csv', index=False)

In [13]:
# csv파일 활용 전 전처리

import pandas as pd

# CSV 파일 읽기, 칼럼 이름이 있는 줄을 지정
try:
    df = pd.read_csv('original/recognition/범죄의_수사단서_2018_2022.csv', header=[0,1])
except:
    print("\n해당하는 파일이 없습니다.")
# '-' 문자를 0으로 대체
df = df.replace('-', 0)

# 년도별로 데이터를 분리하고 각각의 CSV 파일로 저장
for year in range(2018, 2023):
    # 해당 년도의 데이터 선택
    df_year = df.loc[:, df.columns.get_level_values(0).str.startswith(str(year)) | df.columns.get_level_values(0).isin(['죄종별(1)', '죄종별(2)'])].copy()
        
    # 새로운 CSV 파일로 저장
    df_year.to_csv(f'csv/recognition/범죄의_수사단서_{year}.csv', index=False)

In [12]:
import csv
import chardet
import numpy as np
import os
# import matplotlib.pyplot as plt
# from matplotlib import rc
from bokeh.plotting import figure, show
from bokeh.transform import dodge
from bokeh.io import output_notebook, export_png # export_svgs
from bokeh.models import HoverTool, ColumnDataSource

output_notebook()

# rc('font', family='Malgun Gothic')

while True :
    year = input("년도를 입력해주세요(2018년도부터 2022년도까지의 정수값만 입력 가능합니다). > ")
    try:
        if 2018 <= int(year) <= 2022 :
            break
        else :
            print("\n2018년도부터 2022년도까지 정수값만 입력가능며, 한 번에 한 개의 년도만 입력해 주세요.")
    except :
        print("\n해당하는 파일이 없습니다.")

# csv 파일이 ANSI 형식으로 되어있으므로 한글을 읽기 위해서 cp949로 인코딩 함 
f1_recognition = open('범죄_발생부터_인지까지의_기간_'+year+'.csv', 'r', encoding='utf-8') 
f2_arrest = open('범죄_발생부터_검거까지의_기간_'+year+'.csv', 'r', encoding='utf-8')
f3_investigation_clues = open('범죄의_수사단서_'+year+'.csv', 'r', encoding='utf-8')
f4_arrest_clues = open('범죄의_검거단서_'+year+'.csv', 'r', encoding='utf-8')

f1_lines = csv.reader(f1_recognition)
f2_lines = csv.reader(f2_arrest)
f3_lines = csv.reader(f3_investigation_clues)
f4_lines = csv.reader(f4_arrest_clues)

f1_header=['죄종별(1)', '1일 이내', '2일 이내', '3일 이내', '10일 이내', '1개월 이내', '3개월 이내', '3개월 초과']
f2_header=['죄종별(1)', '1일 이내', '2일 이내', '5일 이내', '10일 이내', '1개월 이내', '3개월 이내', '3개월 초과']
f3_header=['죄종별(1)', '현행범', '신고', '미신고']
f3_reporting_header=['죄종별(1)', '피해자신고', '고소', '고발', '자수', '진정투서', '타인신고']
f4_header=['죄종별(1)', '현행범', '피해자신고', '고소', '고발', '자수', '민간체포', '진정투서', \
                       '타인신고', '신문', '불심검문', '탐문정보', '안면식별', '감식자료', '변사체', '범죄수법', \
                       '발자국', '검사지휘', '타기관에서이송', '범행도구발견', '장물', '전과조회', '기타']

# print(f1_header)
# print(f2_header)

# rawdata = open('범죄_발생부터_검거까지의_기간_'+year+'.csv', 'rb').read()
# result = chardet.detect(rawdata)
# print(result)

crime_types = ['강력범죄', '폭력범죄', '지능범죄']

recognition_time = {} # 범죄 발생부터 인지까지의 기간
arrest_time = {} # 범죄 발생부터 검거까지의 기간
investigation_clues = {} # 범죄 수사 단서
arrest_clues = {} # 범죄 검거 단서

for f1_line in f1_lines :
    for crime_type in crime_types:
        if(crime_type in f1_line[0]) and ('소계'in f1_line[1]) :
            recognition_time.update({crime_type:f1_line[2:10]})

# print(recognition_time)
            
for f2_line in f2_lines :
    for crime_type in crime_types:
        if(crime_type in f2_line[0]) and ('소계'in f2_line[1]) :
            arrest_time.update({crime_type:f2_line[2:10]})
        
# print(arrest_time)

for f3_line in f3_lines :
    for crime_type in crime_types:
        if(crime_type in f3_line[0]) and ('소계'in f3_line[1]) :
            investigation_clues.update({crime_type:f3_line[2:12]})

# print(investigation_clues)

for f4_line in f4_lines :
    for crime_type in crime_types:
        if(crime_type in f4_line[0]) and ('소계'in f4_line[1]) :
            arrest_clues.update({crime_type:f4_line[2:23]})
            
# print(arrest_clues)            

f1_bar_data = np.zeros((3,8), dtype = 'float32')
f1_plot_data = np.zeros((3,8), dtype = 'float32')
recognition_date = list(recognition_time.values())
# print(recognition_date)

f2_bar_data = np.zeros((3,8), dtype = 'float32')
f2_plot_data = np.zeros((3,8), dtype = 'float32')
arrest_date = list(arrest_time.values())
# print(arrest_date)

f3_bar_data = np.zeros((3,9), dtype = 'float32')
f3_reporting_bar_data = np.zeros((3,9), dtype = 'float32')
investigation_clues1 = list(investigation_clues.values())

f4_bar_data = np.zeros((3,22), dtype = 'float32')
arrest_clues1 = list(arrest_clues.values())

f1_xaxis = f1_header
f2_xaxis = f2_header
f3_xaxis = f3_header
f3_reporting_xaxis = f3_reporting_header
f4_xaxis = f4_header

del f1_xaxis[0]
del f2_xaxis[0]
del f3_xaxis[0]
del f3_reporting_xaxis[0]
del f4_xaxis[0]

# print(f1_xaxis)

for i in range(len(recognition_time)) :
    crime_rate = 0
    for j in range(8) :
        # print(recognition_date[i][0])
#         print(i)
#         print(j)
        try :
#             print(recognition_date[i][j+1])
            f1_bar_data[i][j] = round((float(recognition_date[i][j+1])/float(recognition_date[i][0])*100),2)
            crime_rate += round((float(recognition_date[i][j+1])/float(recognition_date[i][0])*100),2)
            f1_plot_data[i][j] = crime_rate
        except :
            j=0
            

f1_bar_data = np.delete(f1_bar_data, 7, axis = 1)
f1_plot_data = np.delete(f1_plot_data, 7, axis = 1)

f1_bar_data[0,3] = f1_bar_data[0,2]+f1_bar_data[0,3]
f1_bar_data[1,3] = f1_bar_data[1,2]+f1_bar_data[1,3]
f1_bar_data[2,3] = f1_bar_data[2,2]+f1_bar_data[2,3]

f1_bar_data = np.delete(f1_bar_data, 2, axis = 1)

# print(f1_bar_data)
# print(f1_plot_data)

for i in range(len(arrest_time)) :
    crime_rate = 0
    for j in range(8) :
        try :
            f2_bar_data[i][j] = round((float(arrest_date[i][j+1])/float(arrest_date[i][0])*100),2)
            crime_rate += round((float(arrest_date[i][j+1])/float(arrest_date[i][0])*100),2)
            f2_plot_data[i][j] = crime_rate
        except :
            j=0
            

f2_bar_data = np.delete(f2_bar_data, 7, axis = 1)
f2_plot_data = np.delete(f2_plot_data, 7, axis = 1)

f2_bar_data[0,3] = f2_bar_data[0,2]+f2_bar_data[0,3]
f2_bar_data[1,3] = f2_bar_data[1,2]+f2_bar_data[1,3]
f2_bar_data[2,3] = f2_bar_data[1,2]+f2_bar_data[2,3]

f2_bar_data = np.delete(f2_bar_data, 2, axis = 1)

for i in range(len(investigation_clues)) :
    crime_rate = 0
    for j in range(11) :
        try :
            f3_bar_data[i][j] = round((float(investigation_clues1[i][j+1])/float(investigation_clues1[i][0])*100),2)
            f3_reporting_bar_data[i][j] = round((float(investigation_clues1[i][j+3])/float(investigation_clues1[i][2])*100),2)
        except :
            j=0
            
f3_bar_data = np.delete(f3_bar_data, range(2, 8), axis = 1)
f3_reporting_bar_data = np.delete(f3_reporting_bar_data, range(6,9), axis = 1)

for i in range(len(arrest_clues)) :
    crime_rate = 0
    for j in range(23) :
        try :
            f4_bar_data[i][j] = round((float(arrest_clues1[i][j+1])/float(arrest_clues1[i][0])*100),2)
        except :
            j=0
            
# print(f4_bar_data)

# print(f3_bar_data)
# print(f3_reporting_bar_data)
# print(f2_bar_data[0,2])
# print(f2_bar_data[0,3])


# print(f2_bar_data)
# print(f2_plot_data)

# print(f1_xaxis)
# print(f1_plot_data[0,:])
# print(f1_plot_data[1,:])
# print(f1_plot_data[2,:])

def create_source(f1_xaxis, f2_xaxis, f1_plot_data, f2_plot_data, f1_header, f2_header, crime_type, index):
    source = ColumnDataSource(data=dict(
        x1=f1_xaxis,
        x2=f2_xaxis,
        y1=f1_plot_data[index,:],
        y2=f2_plot_data[index,:],
        term1 = f1_header,
        term2 = f2_header,
        crime_type1=[crime_type]*len(f1_xaxis),
        crime_type2=[crime_type]*len(f2_xaxis),
        desc1 = ['해당 지표는']*len(f2_xaxis),
        desc2 = ['경찰이 인지한 사건에 한하여 검거한 비율을 나타냄']*len(f2_xaxis),
        desc3 = ['경찰이 인지하지 못한 사건에 관한 비율은 알 수 없음']*len(f2_xaxis),
        desc4 = ['']*len(f2_xaxis)
    ))
    return source


# 강력범죄에 대한 ColumnDataSource 생성
source1 = create_source(f1_xaxis, f2_xaxis, f1_plot_data, f2_plot_data, f1_header, f2_header, '강력범죄', 0)

# 폭력범죄에 대한 ColumnDataSource 생성
source2 = create_source(f1_xaxis, f2_xaxis, f1_plot_data, f2_plot_data, f1_header, f2_header, '폭력범죄', 1)

# 지능범죄에 대한 ColumnDataSource 생성
source3 = create_source(f1_xaxis, f2_xaxis, f1_plot_data, f2_plot_data, f1_header, f2_header, '지능범죄', 2)



# 그래프 생성
p = figure(title=year+'년 범죄 발생부터 인지하는 데까지 걸리는 기간(백분율)', x_axis_label='범죄 유형별 경찰 인지 시간', 
           y_axis_label='경찰 인지 누적 비율(%)', x_range=f1_xaxis, width=800, height=500)

# 데이터 추가
p.line(f1_xaxis, f1_plot_data[0,:], legend_label='강력범죄', line_color="#FD8A8A", line_dash="solid", line_width=2)
circle_renderer = p.circle('x1', 'y1', fill_color="#FD8A8A", size=8, source=source1)

p.line(f1_xaxis, f1_plot_data[1,:], legend_label='폭력범죄', line_color="#7EAA92", line_dash="solid", line_width=2)
square_renderer = p.square('x1', 'y1', fill_color="#7EAA92", size=8, source=source2)

p.line(f1_xaxis, f1_plot_data[2,:], legend_label='지능범죄', line_color="#9EA1D4", line_dash="solid", line_width=2)
inverted_triangle_renderer = p.inverted_triangle('x1', 'y1', fill_color="#9EA1D4", size=8, source=source3)


# HoverTool 객체를 생성
hover1 = HoverTool(
    tooltips=[
        ("Types of Criminal Offenses", "@crime_type1"),
        ("Term","@term1"),
        ("Value", "$y{0.00}%"), # 소수점 첫째 자리까지만 표시
    ],
    formatters={
        '@y' : 'printf',  # 'Value' 필드의 형식을 변경
    },
    renderers = [circle_renderer, square_renderer, inverted_triangle_renderer]
)

hover1.renderers = [circle_renderer, square_renderer, inverted_triangle_renderer]

p.add_tools(hover1)

# 사용자의 입력을 받습니다.
save_graph = input(year+'년 범죄 발생부터 인지·검거하는 데까지 걸리는 기간(백분율) 그래프를 저장하시겠습니까? (예/아니오) > ')
print("'예'를 제외한 대답은 그래프를 저장하지 않습니다.")

# 그래프 출력
show(p)

# 그래프 저장 및 저장 여부 출력
if save_graph == '예':
    # 저장할 디렉토리 이름
    directory = "save_graph"
    sub_directory = "recognition_arrest"
    full_path = os.path.join(directory, sub_directory)

    if not os.path.exists(full_path):
        os.makedirs(full_path)
        
    # SVG 파일로 저장
#     p.output_backend = "svg"

    export_png(p, filename=os.path.join(full_path, year+"년 범죄발생부터 인지하는 데까지 걸리는 기간(백분율).png"))
    print("그래프를 저장하였습니다.")

else:
    print("그래프를 저장하지 않습니다.")

# 그래프 생성
p = figure(title=year+'년 범죄발생부터 검거하는 데까지 걸리는 기간(백분율)', x_axis_label='검거 기간', 
           y_axis_label='검거 누적 비율(%)', x_range=f2_xaxis, width=800, height=500)

# 데이터 추가
p.line(f2_xaxis, f2_plot_data[0,:], legend_label='강력범죄', line_color="#FD8A8A", line_dash="solid", line_width=2)
circle_renderer = p.circle('x2', 'y2', fill_color="#FD8A8A", size=8, source=source1)

p.line(f2_xaxis, f2_plot_data[1,:], legend_label='폭력범죄', line_color="#7EAA92", line_dash="solid", line_width=2)
square_renderer = p.square('x2', 'y2', fill_color="#7EAA92", size=8, source=source2)

p.line(f2_xaxis, f2_plot_data[2,:], legend_label='지능범죄', line_color="#9EA1D4", line_dash="solid", line_width=2)
inverted_triangle_renderer = p.inverted_triangle('x2', 'y2', fill_color="#9EA1D4", size=8, source=source3)


# HoverTool 객체를 생성
hover2 = HoverTool(
    tooltips=[
        ("※ Refer","@desc1"),
        ("","@desc2"),
        ("","@desc3"),
        ("","@desc4"),
        ("Types of Criminal Offenses", "@crime_type2"),
        ("Term","@term2"),
        ("Value", "$y{0.00}%"), # 소수점 첫째 자리까지만 표시
    ],
    formatters={
        '@y' : 'printf',  # 'Value' 필드의 형식을 변경
    },
    renderers = [circle_renderer, square_renderer, inverted_triangle_renderer]
)

hover2.renderers = [circle_renderer, square_renderer, inverted_triangle_renderer]

p.add_tools(hover2)

# 그래프 출력
show(p)

# 그래프 저장 여부 출력
if save_graph == '예':
#     print(directory)
    # SVG 파일로 저장
#     p.output_backend = "svg"

    export_png(p, filename=os.path.join(full_path, year+"년 범죄발생부터 검거하는 데까지 걸리는 기간(백분율).png"))
    print("그래프를 저장하였습니다.")
else:
    print("그래프를 저장하지 않습니다.\n")

del f1_xaxis[2]
del f2_xaxis[2]

# 막대그래프에 슬 호버툴을 정형화하기 위하여 함수 사용
def add_hover_tool(plot, source, field_names):
    # 툴팁을 생성
    tooltips = [(name, "@" + name) for name in field_names]
    tooltips.append(("Value", "$y{0.00}%"))  # 소수점 첫째 자리까지만 표시

    # HoverTool 객체를 생성
    hover = HoverTool(
        tooltips=tooltips,
        formatters={
            '@y' : 'printf',  # 'Value' 필드의 형식을 변경
        },
    )

    # 그래프에 HoverTool을 추가합니다.
    plot.add_tools(hover)

save_graph = input(year+'강력범죄, 폭력범죄, 지능범죄 인지,검거 분포율 그래프를 저장하시겠습니까? (예/아니오) > ')
print("'예'를 제외한 대답은 그래프를 저장하지 않습니다.")

# ColumnDataSource 생성
source = ColumnDataSource(data=dict(x=f1_xaxis, y1=f1_bar_data[0,:], y2=f2_bar_data[0,:]))

# 그래프 생성
p = figure(title=year+'년 강력범죄 인지,검거 분포율(%)', x_axis_label='기간', 
           y_axis_label='인지, 검거 분포 비중(%)', x_range=f1_xaxis, width=800, height=500)

# 데이터 추가
p.vbar(x=dodge('x', -0.15, range=p.x_range), top='y1', source=source, legend_label='범죄 발생 인지율', fill_color="#FD8A8A", width=0.25)
p.vbar(x=dodge('x', 0.15, range=p.x_range), top='y2', source=source, legend_label='범죄자 검거율', fill_color="#9EA1D4", width=0.25)

# 그래프 출력
show(p)

# 그래프 저장 및 저장 여부 출력
if save_graph == '예':
    # 저장할 디렉토리 이름
    directory = "save_graph"
    sub_directory = "violent_crime"
    full_path = os.path.join(directory, sub_directory)

    if not os.path.exists(full_path):
        os.makedirs(full_path)
        
    # SVG 파일로 저장
#     p.output_backend = "svg"

    export_png(p, filename=os.path.join(full_path, year+"년 강력범죄 인지,검거 분포율.png"))
    print("그래프를 저장하였습니다.")

else:
    print("그래프를 저장하지 않습니다.")
    
# ColumnDataSource 생성
source = ColumnDataSource(data=dict(x=f1_xaxis, y1=f1_bar_data[1,:], y2=f2_bar_data[1,:]))

# 그래프 생성
p = figure(title=year+'년 폭력범죄 인지,검거 분포율(%)', x_axis_label='기간', 
           y_axis_label='인지, 검거 분포 비중(%)', x_range=f1_xaxis, width=800, height=500)

# 데이터 추가
p.vbar(x=dodge('x', -0.15, range=p.x_range), top='y1', source=source, legend_label='범죄 발생 인지율', fill_color="#FD8A8A", width=0.25)
p.vbar(x=dodge('x', 0.15, range=p.x_range), top='y2', source=source, legend_label='범죄자 검거율', fill_color="#9EA1D4", width=0.25)

# 그래프 출력
show(p)

if save_graph == '예':
    # 저장할 디렉토리 이름
    directory = "save_graph"
    sub_directory = "assault_crime"
    full_path = os.path.join(directory, sub_directory)

    if not os.path.exists(full_path):
        os.makedirs(full_path)
        
    # SVG 파일로 저장
#     p.output_backend = "svg"

    export_png(p, filename=os.path.join(full_path, year+"년 폭력범죄 인지,검거 분포율.png"))
    print("그래프를 저장하였습니다.")

else:
    print("그래프를 저장하지 않습니다.")

# ColumnDataSource 생성
source = ColumnDataSource(data=dict(x=f1_xaxis, y1=f1_bar_data[2,:], y2=f2_bar_data[2,:]))

# 그래프 생성
p = figure(title=year+'년 지능범죄 인지,검거 분포율(%)', x_axis_label='기간', 
           y_axis_label='인지, 검거 분포 비중(%)', x_range=f1_xaxis, width=800, height=500)

# 데이터 추가
p.vbar(x=dodge('x', -0.15, range=p.x_range), top='y1', source=source, legend_label='범죄 발생 인지율', fill_color="#FD8A8A", width=0.25)
p.vbar(x=dodge('x', 0.15, range=p.x_range), top='y2', source=source, legend_label='범죄자 검거율', fill_color="#9EA1D4", width=0.25)

# 그래프 출력
show(p)

if save_graph == '예':
    # 저장할 디렉토리 이름
    directory = "save_graph"
    sub_directory = "intelligent_crime"
    full_path = os.path.join(directory, sub_directory)

    if not os.path.exists(full_path):
        os.makedirs(full_path)
        
    # SVG 파일로 저장
#     p.output_backend = "svg"

    export_png(p, filename=os.path.join(full_path, year+"년 지능범죄 인지,검거 분포율.png"))
    print("그래프를 저장하였습니다.")

else:
    print("그래프를 저장하지 않습니다.\n")

save_graph = input(year+'년 수사단서,수사단서(신고), 검거단서 그래프를 저장하시겠습니까? (예/아니오) > ')
print("'예'를 제외한 대답은 그래프를 저장하지 않습니다.")    

# ColumnDataSource 생성
source = ColumnDataSource(data=dict(x=f3_xaxis, y1=f3_bar_data[0,:], y2=f3_bar_data[1,:],y3=f3_bar_data[2,:]))

# 그래프 생성
p = figure(title=year+'년 범죄 수사 단서 분포율(%)', x_axis_label='수사 단서', 
           y_axis_label='수사 단서 분포 비중(%)', x_range=f3_xaxis, width=800, height=500)

# 데이터 추가
p.vbar(x=dodge('x', -0.3, range=p.x_range), top='y1', source=source, legend_label='강력범죄', fill_color="#FD8A8A", width=0.25)
p.vbar(x=dodge('x', 0, range=p.x_range), top='y2', source=source, legend_label='폭력범죄', fill_color="#7EAA92", width=0.25)
p.vbar(x=dodge('x', 0.3, range=p.x_range), top='y3', source=source, legend_label='지능범죄', fill_color="#9EA1D4", width=0.25)

# 그래프 출력
show(p)

if save_graph == '예':
    # 저장할 디렉토리 이름
    directory = "save_graph"
    sub_directory = "Investigation_clues"
    full_path = os.path.join(directory, sub_directory)

    if not os.path.exists(full_path):
        os.makedirs(full_path)
        
    # SVG 파일로 저장
#     p.output_backend = "svg"

    export_png(p, filename=os.path.join(full_path, year+"년 수사 단서.png"))
    print("그래프를 저장하였습니다.")

else:
    print("그래프를 저장하지 않습니다.")

# ColumnDataSource 생성
source = ColumnDataSource(data=dict(x=f3_reporting_xaxis, y1=f3_reporting_bar_data[0,:], y2=f3_reporting_bar_data[1,:],y3=f3_reporting_bar_data[2,:]))

# 그래프 생성
p = figure(title=year+'년 범죄 신고 분포율(%)', x_axis_label='신고 종류', 
           y_axis_label='수사 신고 분포 비중(%)', x_range=f3_reporting_xaxis,width=800, height=500)

# 데이터 추가
p.vbar(x=dodge('x', -0.3, range=p.x_range), top='y1', source=source, legend_label='강력범죄', fill_color="#FD8A8A", width=0.25)
p.vbar(x=dodge('x', 0, range=p.x_range), top='y2', source=source, legend_label='폭력범죄', fill_color="#7EAA92", width=0.25)
p.vbar(x=dodge('x', 0.3, range=p.x_range), top='y3', source=source, legend_label='지능범죄', fill_color="#9EA1D4", width=0.25)

# 그래프 출력
show(p)

if save_graph == '예':
    export_png(p, filename=os.path.join(full_path, year+"년 수사(신고) 단서.png"))
    print("그래프를 저장하였습니다.")

else:
    print("그래프를 저장하지 않습니다.")
# ColumnDataSource 생성
source = ColumnDataSource(data=dict(x=f4_xaxis, y1=f4_bar_data[0,:], y2=f4_bar_data[1,:],y3=f4_bar_data[2,:]))

# 그래프 생성
p = figure(title=year+'년 범죄자 검거 단서 분포율(%)', x_axis_label='검거 단서', 
           y_axis_label='검거 단서 분포 비중(%)', x_range=f4_xaxis, width=800, height=500)

# 데이터 추가
p.vbar(x=dodge('x', -0.3, range=p.x_range), top='y1', source=source, legend_label='강력범죄', fill_color="#FD8A8A", width=0.25)
p.vbar(x=dodge('x', 0, range=p.x_range), top='y2', source=source, legend_label='폭력범죄', fill_color="#7EAA92", width=0.25)
p.vbar(x=dodge('x', 0.3, range=p.x_range), top='y3', source=source, legend_label='지능범죄', fill_color="#9EA1D4", width=0.25)

# x축 레이블을 45도 회전
p.xaxis.major_label_orientation = "vertical"

# 그래프 출력
show(p)

if save_graph == '예':
    # 저장할 디렉토리 이름
    directory = "save_graph"
    sub_directory = "arrest_clue"
    full_path = os.path.join(directory, sub_directory)

    if not os.path.exists(full_path):
        os.makedirs(full_path)
        
    # SVG 파일로 저장
#     p.output_backend = "svg"
    #PNG 파일로 저장
    export_png(p, filename=os.path.join(full_path, year+"년 검거 단서.png"))
    print("그래프를 저장하였습니다.")

else:
    print("그래프를 저장하지 않습니다.")

f1_recognition.close()
f2_arrest.close()
f3_investigation_clues.close()
f4_arrest_clues.close()

Loading BokehJS ...

년도를 입력해주세요(2018년도부터 2022년도까지의 정수값만 입력 가능합니다). > dfg

해당하는 파일이 없습니다.
년도를 입력해주세요(2018년도부터 2022년도까지의 정수값만 입력 가능합니다). > dfg

해당하는 파일이 없습니다.
년도를 입력해주세요(2018년도부터 2022년도까지의 정수값만 입력 가능합니다). > dfg

해당하는 파일이 없습니다.
년도를 입력해주세요(2018년도부터 2022년도까지의 정수값만 입력 가능합니다). > dfg

해당하는 파일이 없습니다.
년도를 입력해주세요(2018년도부터 2022년도까지의 정수값만 입력 가능합니다). > 2018
2018년 범죄 발생부터 인지·검거하는 데까지 걸리는 기간(백분율) 그래프를 저장하시겠습니까? (예/아니오) > fgd
'예'를 제외한 대답은 그래프를 저장하지 않습니다.


그래프를 저장하지 않습니다.


그래프를 저장하지 않습니다.

2018강력범죄, 폭력범죄, 지능범죄 인지,검거 분포율 그래프를 저장하시겠습니까? (예/아니오) > g
'예'를 제외한 대답은 그래프를 저장하지 않습니다.


그래프를 저장하지 않습니다.


그래프를 저장하지 않습니다.


그래프를 저장하지 않습니다.

2018년 수사단서,수사단서(신고), 검거단서 그래프를 저장하시겠습니까? (예/아니오) > df
'예'를 제외한 대답은 그래프를 저장하지 않습니다.


그래프를 저장하지 않습니다.


그래프를 저장하지 않습니다.


그래프를 저장하지 않습니다.
